# HRD dropsonde read and plot with MetPy

#### This was the web site fetch command, but I need to narrow it down 
wget -r https://seb.noaa.gov/pub/acdata/2021/AVAPS/20210828I1/ASPEN_DATA/*

View an example at https://seb.noaa.gov/pub/acdata/2021/AVAPS/20210828I1/ASPEN_DATA/D20210828_091908QC.frd

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import metpy

In [26]:
filename = '~/Box/MPO624_data/VP_hurr_sondes/20210828I1/ASPEN_DATA/D20210828_093202QC.frd'
df = pd.read_csv(filename, skiprows=20, delim_whitespace=True)
df
# aww shoot, there are spaces in the header line like between P and (mb) so it looks like 2 things
# gonna have to manually make column names

,IX,t,(s),P,(mb),T,(C),RH,(%),Z,...,ZW,(m).1,FP,FT,FH,FW,LAT,(N),LON,(E)
0,1,0.00,-999.0,-999.00,-999.0,-999,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.25,-999.0,-999.00,-999.0,-999,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0.50,-999.0,-999.00,89.9,-999,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.75,-999.0,-999.00,-999.0,-999,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1.00,-999.0,-999.00,89.4,-999,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,1139,284.50,1003.5,27.51,81.0,14,29,15.56,-7.49,-13.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1139,1140,284.75,-999.0,-999.00,-999.0,-999,28,15.11,-7.21,-13.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,1141,285.00,1004.0,27.57,80.8,9,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,1142,285.25,-999.0,-999.00,-999.0,-999,-999,-999.00,-999.00,-999.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Have to set -999 to NaN

df['T'].values

array([-999, -999, -999, ...,    9, -999,    0])

hi